In [1]:
import glob
import pandas as pd
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "src")))
from utils import Utils
from reports import Reports
import matplotlib.pyplot as plt
from overlays import Overlays
from utils import Utils
import numpy as np

In [2]:
all_tiled_image_paths, all_tiled_label_paths = glob.glob(r"D:\datasets\tiled\images\*.png"), glob.glob(r"D:\datasets\tiled\labels\*.txt")
print(len(all_tiled_image_paths))
assert len(all_tiled_image_paths) == len(all_tiled_label_paths), "Mismatch between image and label counts."

0


In [3]:
# filter for images with n_fish >= n images
df_run12_meta_nf = pd.read_csv(r"Z:\__Organized_Directories_InProgress\GobyFinderDatasets\AUV_datasets\runs\run12\run12_metadata.csv", index_col=0, low_memory=False)
df_run12_meta_nf_usable = df_run12_meta_nf[df_run12_meta_nf.Usability == "Usable"]
meta_df = df_run12_meta_nf_usable
df_abs2_meta_nf = meta_df[(meta_df.n_fish >= 0)  & (meta_df.imh > 2176)]
df_abs1_meta_nf = meta_df[(meta_df.n_fish >= 0)  & (meta_df.imh == 2176)]
abiss2_len = len(df_abs2_meta_nf) 
abiss1_len = len(df_abs1_meta_nf)  
# export_paths(df_abs1_meta_nf[df_abs1_meta_nf.split == "test"], run13_folder, subfolder="abiss1_test")
print("usable Abiss1 images:", abiss1_len)
print("usable Abiss2 images:", abiss2_len) #usable Abiss2 images: 14225
few_fish_df = df_abs2_meta_nf[(df_abs2_meta_nf.n_fish < 2)]
new_path = r"Z:\__Organized_Directories_InProgress\GobyFinderDatasets\AUV_datasets\one_fish\images"
output_dir = r"Z:\__Organized_Directories_InProgress\GobyFinderDatasets\AUV_datasets\one_fish\overlays"
# Overlays.plot_label_overlays(few_fish_df.image_path, few_fish_df.label_path, output_dir, overwrite=False)

usable Abiss1 images: 19299
usable Abiss2 images: 14225


In [4]:
all_images_df = df_abs2_meta_nf[(df_abs2_meta_nf.n_fish >= 2)]
# Define constants for the batching process
BATCH_SIZE = 100
# Placeholder for your run folder variable
innodata_update = r"Z:\__AdvancedTechnologyBackup\04_ProjectData\Innodata_2025\Goby"

# Step 1: Get the initial sample
initial_relabel = all_images_df.sample(n=BATCH_SIZE, random_state=123) # Use BATCH_SIZE here

# Step 2: Export the initial sample batch
name_initial = "2025_goby_relabel_initial_sample"
Utils.export_paths(initial_relabel.image_path, initial_relabel.label_path, innodata_update, os.path.join(name_initial, "full"))
tiled_images, tiled_labels  = Utils.list_tiled_set(initial_relabel.Filename.values, all_tiled_image_paths, all_tiled_label_paths)
Utils.export_paths(tiled_images, tiled_labels, innodata_update, os.path.join(name_initial, "tiled"))
# Step 3: Create a DataFrame with the remaining images
# We do this by getting the indices of the initial sample and
# selecting all rows from the original DataFrame whose indices are NOT in that sample.
# Using .copy() is good practice here to avoid SettingWithCopyWarning later
remaining_images_df = all_images_df.drop(initial_relabel.index).copy()

# Now, we proceed with the rest of the batches on the remaining images
# Use a while loop because the DataFrame size changes
batch_number = 1
while len(remaining_images_df) > 0:
    # Determine the size of the current batch
    # This handles the case where the remaining images are less than BATCH_SIZE
    current_batch_size = min(BATCH_SIZE, len(remaining_images_df))

    # Get a random sample of the required size from the remaining data
    # Do NOT set random_state here if you want different random samples each time
    subset = remaining_images_df.sample(n=current_batch_size)

    # Export the current random sample batch
    name = f"2025_goby_relabel_batch_{batch_number:02d}"
    Utils.export_paths(subset.image_path, subset.label_path, innodata_update, os.path.join(name, "full"))
    tiled_images, tiled_labels  = Utils.list_tiled_set(subset.Filename.values, all_tiled_image_paths, all_tiled_label_paths)
    Utils.export_paths(tiled_images, tiled_labels, innodata_update, os.path.join(name, "tiled"))

    # Step 4: CRITICAL - Remove the sampled subset from the remaining pool
    # The .drop() function ensures the next iteration samples from a non-redundant pool.
    remaining_images_df.drop(subset.index, inplace=True)
    
    # Increment the batch counter
    batch_number += 1

print(f"Total batches created: {batch_number - 1}")

Total batches created: 69


In [33]:
# Copy Batches
batch = "batch_01"  # Change this to the desired batch number
path = f"D:\\ageglio-1\\gobyfinder_yolov8\\datasets\\AUV_datasets\\innodata_updates\\2025_goby_relabel_{batch}\\tiled"
images, labels = Utils.read_list_txt(os.path.join(path, "images.txt")), Utils.read_list_txt(os.path.join(path, "labels.txt"))
img_folder, lbl_folder = os.path.join(path, "images"), os.path.join(path, "labels")
Utils.copy_files_lst(images, img_folder)
Utils.copy_files_lst(labels, lbl_folder)

In [ ]:
curve_path_0n = r"..\output\validation\detect\test_sept_1fish\test_run13_1n_curves.csv"
support_0n = df_abs2_meta_nf[(df_abs2_meta_nf.n_fish < 2)].shape[0]
curve_path_2n = r"..\output\validation\detect\test_sept_2-3fish\test_run13_2-3n_curves.csv"
support_2n = df_abs2_meta_nf[(df_abs2_meta_nf.n_fish >= 2) & (df_abs2_meta_nf.n_fish <= 3)].shape[0]
curve_path_4n = r"..\output\validation\detect\test_sept_4fish+\test_run13_4n+_curves.csv"
support_4n = df_abs2_meta_nf[(df_abs2_meta_nf.n_fish >= 4)].shape[0]
curve_path_abs2 = r"..\output\validation\detect\test_sept_abiss2\test_run13_abiss2_curves.csv"
support_abs2 = df_abs2_meta_nf.shape[0]
curve_path_abs1 = r"..\output\validation\detect\test_sept_abiss1\test_run13_abiss1_curves.csv"
support_abs1 = df_abs1_meta_nf.shape[0]
df_0n, fmax_0n, cmax_0n, c_eq_0n, pr_eq_0n = Reports.get_metrics(curve_path_0n) 
df_2n, fmax_2n, cmax_2n, c_eq_2n, pr_eq_2n = Reports.get_metrics(curve_path_2n) 
df_4n, fmax_4n, cmax_4n, c_eq_4n, pr_eq_4n = Reports.get_metrics(curve_path_4n) 
dfabs2, fmaxabs2, cmaxabs2, c_eqabs2, pr_eqabs2 = Reports.get_metrics(curve_path_abs2) 
dfabs1, fmaxabs1, cmaxabs1, c_eqabs1, pr_eqabs1 = Reports.get_metrics(curve_path_abs1) 

In [ ]:
plt.plot(df_0n.precision, df_0n.recall, label=f"ABISS2 only 1 fish, {support_0n}, {support_0n/abiss2_tst*100:0.1f}%", color="purple")
plt.plot(df_2n.precision, df_2n.recall, label=f"ABISS2 At 2-3 fish, {support_2n}, {support_2n/abiss2_tst*100:0.1f}%", color="green")
plt.plot(df_4n.precision, df_4n.recall, label=f"ABISS2 At least 4 fish, {support_4n}, {support_4n/abiss2_tst*100:0.1f}%", color="orange")
plt.plot(dfabs1.precision, dfabs1.recall, label=f"ABISS1 Test images, {abiss1_tst}", color="red")
plt.xlabel('Recall') 
plt.ylabel('Precision') 
plt.title('Goby count bins Precision-Recall Curve')
plt.legend() # 1461

In [ ]:
json_path = r"..\datasets\AUV_datasets\run13\initial_relabel\labels.json"
image_path_txt = r"..\datasets\AUV_datasets\run13\initial_relabel\images.txt"
image_lst = pd.read_csv(image_path_txt, header=None)
image_path = image_lst.iloc[31].values[0]
Overlays.plot_coco_boxes(image_path, json_path)

In [ ]:
plt.plot(dfabs2.precision, dfabs2.recall, label=f"ABISS2 Test images, {support_abs2}")
plt.plot(dfabs1.precision, dfabs1.recall, label=f"ABISS1 Test images, {support_abs1}", color="red")
plt.xlabel('Recall') 
plt.ylabel('Precision') 
plt.title('ABISS1 vs ABISS2 Precision-Recall Curve')
plt.legend()

In [ ]:
sample = df_run12_meta_nf[df_run12_meta_nf.collect_id == "20200824_001_Iver3069_ABS1"]
times = sample.Time_s/1.5982e9 # Convert to milliseconds for better readability
types = sample.split
b = len(times)

plt.figure(figsize=(15, 4))
plt.hist(times[types == "train"], bins=b, label='Train', alpha=0.5)
plt.hist(times[types == "test"], bins=b, label='Test', alpha=0.5)
plt.hist(times[types == "validation"], bins=b, label='Val', alpha=0.5)
plt.xlabel('Image Index')
plt.ticklabel_format(style='plain', axis='x')
plt.show()
